# Shared Evaluation (Base + LoRA)

Evaluates base checkpoint and all LoRA adapters from one shared config.


In [ ]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / '.git').exists():
            return candidate
    return start


REPO_ROOT = find_repo_root(Path.cwd().resolve())
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

EVAL_CONFIG_PATH = REPO_ROOT / 'configs/eval/shared_eval.yaml'
EVAL_MODE = 'quick'  # quick | full
EVAL_CONFIG_PATH


In [ ]:
from pipelines.dependencies import check_packages, assert_required_packages

status = check_packages(['torch', 'torchvision', 'yaml', 'safetensors', 'einops', 'tqdm', 'pandas'])
status


In [ ]:
assert_required_packages(['torch', 'torchvision', 'yaml', 'safetensors', 'einops'])
print('Core dependencies look good')


In [ ]:
from pipelines.contracts import EvalConfig

cfg = EvalConfig.from_yaml(EVAL_CONFIG_PATH).eval
cfg


In [ ]:
for dataset_cfg in cfg['datasets']:
    manifest_path = REPO_ROOT / dataset_cfg['manifest']
    if not manifest_path.exists():
        raise FileNotFoundError(f'Missing manifest: {manifest_path}')
print('All manifests found')


In [ ]:
cmd = [
    sys.executable,
    str(REPO_ROOT / 'scripts/pipelines/eval_from_config.py'),
    '--config',
    str(EVAL_CONFIG_PATH),
    '--mode',
    EVAL_MODE,
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


In [ ]:
import pandas as pd

metrics_dir = REPO_ROOT / cfg.get('output_dir', 'results/shared_eval')
metrics_csv = metrics_dir / 'metrics.csv'
metrics_json = metrics_dir / 'metrics.json'

print('Metrics CSV :', metrics_csv)
print('Metrics JSON:', metrics_json)

if metrics_csv.exists():
    df = pd.read_csv(metrics_csv)
    display(df)
else:
    print('metrics.csv not found')


## Done checklist

- [ ] All dataset manifests exist
- [ ] Quick eval completed without exception
- [ ] metrics.json and metrics.csv were generated
- [ ] Full eval run completed after all training jobs
